# Projeto 3 - Ranting de filmes (IMDB)

#### Por: Marcelo Lisboa de Castro Reis, Tiago Niemeyer Bergamo  e Bruno Kaczelnik- 2C Engenharia Insper

A partir do elenco, diretores, gênero do filme, ratings de filmes da mesma “linha”, bilheteria, pretendemos avaliar qual será a nota que um filme receberá. 

**O nosso projeto pode ser encontrado no seguinte GitHub:** https://github.com/Marcelolcr/Projeto-1---Ciencia-de-Dados.git

**Referências (Link do dataset):** 
- https://www.kaggle.com/carolzhangdc/imdb-5000-movie-dataset
- https://datasets.imdbws.com/


In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import math as math
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
#Criando o dataframe Treinamento
imdb = pd.read_csv('movie_metadata.csv')
imdb = imdb.set_index('movie_title')
imdb = imdb.drop(['aspect_ratio','movie_imdb_link','facenumber_in_poster'], 1)
imdb#sort_values(by='gross', ascending=False).head(10)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,plot_keywords,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,movie_facebook_likes
movie_title,,,,,,,,,,,,,,,,,,,,,
Avatar,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,avatar|future|marine|native|paraplegic,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,33000
Pirates of the Caribbean: At World's End,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,goddess|marriage ceremony|marriage proposal|pi...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,0
Spectre,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,bomb|espionage|sequel|spy|terrorist,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,85000
The Dark Knight Rises,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,deception|imprisonment|lawlessness|police offi...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,164000
Star Wars: Episode VII - The Force Awakens,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,0
John Carter,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,alien|american civil war|male nipple|mars|prin...,738.0,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,24000
Spider-Man 3,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,sandman|spider man|symbiote|venom|villain,1902.0,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,0
Tangled,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,17th century|based on fairy tale|disney|flower...,387.0,English,USA,PG,260000000.0,2010.0,553.0,7.8,29000
Avengers: Age of Ultron,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,artificial intelligence|based on comic book|ca...,1117.0,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,118000


In [3]:
#Limpando os dados
netflix['Treinamento'] = netflix['Treinamento'].str.replace(',','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace(':','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('|','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('.','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('"','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('”','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('“','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace(';','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('!',' !')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('?',' ?')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('(','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('+','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace(')','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('\n','')
netflix['Treinamento'] = netflix['Treinamento'].str.replace('/','') #
netflix['Treinamento'] = netflix['Treinamento'].str.strip()
netflix['Treinamento'] = netflix['Treinamento'].str.lower()
#netflix.head()

In [4]:
#Separando os dados já classificados em dois grupos: relevantes e irrelevantes
pal_relevantes = netflix[netflix['R ou I']=='Relevante']
pal_irrelevantes = netflix[netflix['R ou I']=='Irrelevante']

netflix_lista_palavras_rel = pal_relevantes['Treinamento'].str.split()
netflix_lista_palavras_irrel = pal_irrelevantes['Treinamento'].str.split()

#Criando novos DataFrames para cada grupo e limpando mais 
Novo_DF_R = pd.DataFrame(np.array(netflix_lista_palavras_rel), columns=['Relevantes'])
Novo_DF_I = pd.DataFrame(np.array(netflix_lista_palavras_irrel), columns=['Irrelevantes'])

for linha in Novo_DF_R['Relevantes']:
    for p in linha:
        if '@' in p:
            linha.remove(p)
        if 'https'in p:
            linha.remove(p)
        if ' ' in p:
            linha.remove(p)
            
for l in Novo_DF_I['Irrelevantes']:
    for pal in l:
        if '@' in pal:
            l.remove(pal)
        if 'https' in pal:
            l.remove(pal)
        if ' ' in pal:
            l.remove(pal)
            
#Probabilidades totais
prob_rel = len(pal_relevantes)/len(netflix)
prob_irrel = len(pal_irrelevantes)/len(netflix)
#Novo_DF_I.head()

In [5]:
#Colocando todas as palavras em uma mesma lista
palavras_R = []
for pal in netflix_lista_palavras_rel:
    for p in pal:
        palavras_R.append(p)

palavras_I = []
for pal in netflix_lista_palavras_irrel:
    for p in pal:
        palavras_I.append(p)

#Tirando os usernames e links da lista
for pal in palavras_R:
    if '@' in pal:
        palavras_R.remove(pal)
    if "https" in pal:
        palavras_R.remove(pal)
        
for pal in palavras_I:
    if '@' in pal:
        palavras_I.remove(pal)
    if "https" in pal:
        palavras_I.remove(pal)

#Calculando o total de palavras diferentes na nossa base Treinamento
PALAVRAS_RI = []
for e in palavras_R:
    if e not in PALAVRAS_RI:
        PALAVRAS_RI.append(e)
for e in palavras_I:
    if e not in PALAVRAS_RI:
        PALAVRAS_RI.append(e)

print(len(PALAVRAS_RI))

#Claculando o numero de palavras relevantes e irrelevantes em cada uma das listas
print(len(palavras_R))
print(len(palavras_I))

1166
1728
3281


In [6]:
#Criando lista de tweets agora
Lista_tweets_R = []
for listas in Novo_DF_R["Relevantes"]:
    Lista_tweets_R.append(listas)
#print(Lista_tweets_R)

Lista_tweets_I = []
for listas in Novo_DF_I["Irrelevantes"]:
    Lista_tweets_I.append(listas)

In [7]:
#Frequencia de palavras relevantes 
D_Relevante = {}
for e in palavras_R:
    if e not in D_Relevante:
        D_Relevante[e] = 1
    elif e in D_Relevante:
        D_Relevante[e] += 1

#Probabilidade de palavras relevantes (não será necessário com o laplace)
PROB_1000_R = {}
for e in D_Relevante:
    probabilidades = D_Relevante[e]/len(palavras_R)
    PROB_1000_R[e] = probabilidades
#print(PROB_1000_R)

In [8]:
#Frequencia de palavras irrelevantes
D_Irrelevante = {}
for e in palavras_I:
    if e not in D_Irrelevante:
        D_Irrelevante[e] = 1
    elif e in D_Irrelevante:
        D_Irrelevante[e] += 1

#Probabilidade de palavras relevantes (não será necessário com o laplace)
PROB_1000_I = {}
for e in D_Irrelevante:
    probabilidades = D_Irrelevante[e]/len(palavras_I)
    PROB_1000_I[e] = probabilidades
#print(PROB_1000_I)

In [9]:
#Probabilidade do tweet ser relevante (não será necessário com o laplace)
Prob_tweet_R = []
pro = 1
for e in Lista_tweets_R:
    for pal in PROB_1000_R:
        if pal in e:
            pro = pro * PROB_1000_R[pal]
    Prob_tweet_R.append(pro)#/len(e))
    pro = 1

#print(Lista_tweets_R)
#print(Prob_tweet_R)

In [10]:
#Probabilidade do tweet ser irrelevante (não será necessário com o laplace)
Prob_tweet_I = []
pro = 1
for e in Lista_tweets_I:
    for pal in PROB_1000_I:
        if pal in e:
            pro = pro * PROB_1000_I[pal]
    Prob_tweet_I.append(pro)#/len(e))
    pro = 1
    
#print(Lista_tweets_I)
#print(len(Prob_tweet_I))

In [11]:
#Fazendo Laplace e testando inicialmente com tweets relevantes do treinamento
PROB_R_LAPLACE = []
PROB_I_LAPLACE = []

for e in Lista_tweets_R:
    
    pro = 1
    for pal in e:
        if pal in D_Relevante.keys():
            pro = pro * (D_Relevante[pal] + 1)/(len(palavras_R)+len(PALAVRAS_RI))
        else:
            pro = pro * (1/(len(palavras_R)+len(PALAVRAS_RI)))        
    P1 = pro * prob_rel
    PROB_R_LAPLACE.append(P1)
    pro = 1

    PRO = 1
    for p in e:
        if p in D_Irrelevante.keys():
            PRO = PRO * (D_Irrelevante[p] + 1)/(len(palavras_I)+len(PALAVRAS_RI))
        else:
            PRO = PRO * (1/(len(palavras_R)+len(PALAVRAS_RI))) 
    P2 = prob_irrel * PRO
    PROB_I_LAPLACE.append(P2)
    PRO = 1
    
relevantes_positivos = []
relevantes_negativos = []

for i in range(0,len(PROB_R_LAPLACE)):
    if PROB_R_LAPLACE[i]>PROB_I_LAPLACE[i]:
        relevantes_positivos.append ("OK")
    else:
        relevantes_negativos.append ("OK")
        
print('Dos 105 tweets marcados como relevantes, foram classificados como: ')      
print('Relevantes: {0} tweets'.format(len(relevantes_positivos))) 
print('Irrelevantes: {0} tweets'.format(len(relevantes_negativos)))

Dos 105 tweets marcados como relevantes, foram classificados como: 
Relevantes: 97 tweets
Irrelevantes: 8 tweets


In [19]:
#Fazendo Laplace e testando inicialmente com tweets irrelevantes do treinamento
PROB2_R_LAPLACE = []
PROB2_I_LAPLACE = []

for e in Lista_tweets_I:
    
    prob = 1
    for pal in e:
        if pal in D_Relevante:
            prob = prob * (D_Relevante[pal] + 1)/(len(palavras_R)+len(PALAVRAS_RI))
        else:
            prob = prob * (1/(len(palavras_R)+len(PALAVRAS_RI)))
    P1 = prob_rel * prob
    PROB2_R_LAPLACE.append(P1)
    prob = 1    
    
    PRO = 1
    for p in e:
        if p in D_Irrelevante:
            PRO = PRO * (D_Irrelevante[p] + 1)/(len(palavras_I)+len(PALAVRAS_RI))
        else:
            PRO = PRO * (1/(len(palavras_I)+len(PALAVRAS_RI)))
    P2 = prob_irrel * PRO
    PROB2_I_LAPLACE.append(P2)
    PRO = 1        

irrelevantes_positivos = []
irrelevantes_negativos = []

for i in range(0,len(PROB2_I_LAPLACE)):
    if PROB2_I_LAPLACE[i]>PROB2_R_LAPLACE[i]:
        irrelevantes_positivos.append ("OK")
    else:
        irrelevantes_negativos.append ("OK")

print('Dos 195 tweets marcados como irrelevantes, foram classificados como: ')
print('Irrelevantes: {0} tweets'.format(len(irrelevantes_positivos)))    
print('Relevantes: {0} tweets'.format(len(irrelevantes_negativos)))
#len(PROB2_I_LAPLACE)

Dos 195 tweets marcados como irrelevantes, foram classificados como: 
Irrelevantes: 192 tweets
Relevantes: 3 tweets


## Testando os dados agora com a base "Teste":

In [13]:
#Importando, limpando, dividindo em relevante e irrelevante, criando os dataframes, ... como com a base "Treinamento"
teste = pd.read_excel('netflix_´p2.xlsx', sheet_name=1)

teste['Teste'] = teste['Teste'].str.replace(',','')
teste['Teste'] = teste['Teste'].str.replace(':','')
teste['Teste'] = teste['Teste'].str.replace('|','')
teste['Teste'] = teste['Teste'].str.replace('.','')
teste['Teste'] = teste['Teste'].str.replace('"','')
teste['Teste'] = teste['Teste'].str.replace('”','')
teste['Teste'] = teste['Teste'].str.replace('“','')
teste['Teste'] = teste['Teste'].str.replace(';','')
teste['Teste'] = teste['Teste'].str.replace('!',' !')
teste['Teste'] = teste['Teste'].str.replace('?',' ?')
teste['Teste'] = teste['Teste'].str.replace('(','')
teste['Teste'] = teste['Teste'].str.replace('+','')
teste['Teste'] = teste['Teste'].str.replace(')','')
teste['Teste'] = teste['Teste'].str.replace('\n','')
teste['Teste'] = teste['Teste'].str.replace('/','')
teste['Teste'] = teste['Teste'].str.strip()
teste['Teste'] = teste['Teste'].str.lower()

teste_relevantes = teste[teste['R ou I']=='Relevante']
teste_irrelevantes = teste[teste['R ou I']=='Irrelevante']

teste_lista_palavras_rel = teste_relevantes['Teste'].str.split()
teste_lista_palavras_irrel = teste_irrelevantes['Teste'].str.split()

teste_R = pd.DataFrame(np.array(teste_lista_palavras_rel), columns=['Relevantes'])
teste_I = pd.DataFrame(np.array(teste_lista_palavras_irrel), columns=['Irrelevantes'])

for linha in teste_R['Relevantes']:
    for p in linha:
        if '@' in p:
            linha.remove(p)
        if 'https'in p:
            linha.remove(p)
        if ' ' in p:
            linha.remove(p)
            
for li in teste_I['Irrelevantes']:
    for palo in li:
        if '@' in palo:
            li.remove(palo)
        if ' ' in palo:
            li.remove(palo)
            
Tweets_teste_R = []
for listas in teste_R["Relevantes"]:
    Tweets_teste_R.append(listas)
Tweets_teste_I = []
for listas in teste_I["Irrelevantes"]:
    Tweets_teste_I.append(listas)

#print(len(Tweets_teste_I))
#print(len(Tweets_teste_R))
len(teste)

200

In [14]:
#Usando Laplace para calcular, a partir do nosso banco de dados, a probabilidade dos tweets relevantes "Testes" serem relevantes
R_LAPLACE = []
I_LAPLACE = []

for e in Tweets_teste_R:
    
    pro = 1
    for pal in e:
        if pal in D_Relevante.keys():
            pro = pro * (D_Relevante[pal] + 1)/(len(palavras_R)+len(PALAVRAS_RI))
        else:
            pro = pro * (1/(len(palavras_R)+len(PALAVRAS_RI))) 
        
    
    P1 = pro * prob_rel
    R_LAPLACE.append(P1)
    pro = 1

    PRO = 1
    for p in e:
        if p in D_Irrelevante.keys():
            PRO = PRO * (D_Irrelevante[p] + 1)/(len(palavras_I)+len(PALAVRAS_RI))
        else:
            PRO = PRO * (1/(len(palavras_R)+len(PALAVRAS_RI))) 
    P2 = prob_irrel * PRO
    I_LAPLACE.append(P2)
    PRO = 1

    
Positivos_verdadeiros = []
Negativos_falsos = []

for i in range(0,len(R_LAPLACE)):
    if R_LAPLACE[i]>I_LAPLACE[i]:
        Positivos_verdadeiros.append ("OK")
    else:
        Negativos_falsos.append ("OK")
        
pos_ved = len(Positivos_verdadeiros)/len(teste)
neg_fal = len(Negativos_falsos)/len(teste)

print('Dos tweets já marcados como relevantes, a probabilidade destes serem relevantes (1ªprob) e irrelevantes (2ªprob) são: ')
print('{0} % e'.format((len(Positivos_verdadeiros)/len(R_LAPLACE))*100)) #marcados como rele sendo rele #positivos verdadeiros
print('{0} %'.format((len(Negativos_falsos)/len(R_LAPLACE))*100)) #marcados como irre sendo rele #negativos falsos

Dos tweets já marcados como relevantes, a probabilidade destes serem relevantes (1ªprob) e irrelevantes (2ªprob) são: 
62.5 % e
37.5 %


In [15]:
#Como no anterior, com o nosso banco de dados, calculamos a probabilidade dos tweets "Testes" irrelevantes serem irrelevantes
R2_LAPLACE = []
I2_LAPLACE = []

for e in Tweets_teste_I:
    
    pro = 1
    for pal in e:
        if pal in D_Relevante.keys():
            pro = pro * (D_Relevante[pal] + 1)/(len(palavras_R)+len(PALAVRAS_RI))
        else:
            pro = pro * (1/(len(palavras_R)+len(PALAVRAS_RI))) 
          
    P1 = pro * prob_rel
    R2_LAPLACE.append(P1)
    pro = 1

    PRO = 1
    for p in e:
        if p in D_Irrelevante.keys():
            PRO = PRO * (D_Irrelevante[p] + 1)/(len(palavras_I)+len(PALAVRAS_RI))
        else:
            PRO = PRO * (1/(len(palavras_R)+len(PALAVRAS_RI))) 
    P2 = prob_irrel * PRO
    I2_LAPLACE.append(P2)
    PRO = 1

    
Negativos_verdadeiros = []
Positivos_falsos = []

for i in range(0,len(I2_LAPLACE)):
    if I2_LAPLACE[i]>R2_LAPLACE[i]:
        Negativos_verdadeiros.append ("OK")
    else:
        Positivos_falsos.append ("OK")
    
neg_ved = len(Negativos_verdadeiros)/len(teste)
pos_fal = len(Positivos_falsos)/len(teste)

print('Dos tweets já marcados como irrelevantes, a probabilidade destes serem irrelevantes (1ªprob) e relevantes (2ªprob) são: ')
print('{0} % e'.format((len(Negativos_verdadeiros)/len(I2_LAPLACE))*100)) #marcado como irre sendo irre #negativos verdadeiros
print('{0} %'.format((len(Positivos_falsos)/len(I2_LAPLACE))*100)) #marcado como rele sendo irre #positivos falsos

Dos tweets já marcados como irrelevantes, a probabilidade destes serem irrelevantes (1ªprob) e relevantes (2ªprob) são: 
93.75 % e
6.25 %


Testando também com um tweet aleatório qualquer:

In [16]:
ahoi = ['netflix', 'americano', 'é', 'melhor']
pb = []
pro = 1
for pala in ahoi:
    
    if pala in D_Relevante.keys():
        pro = pro * (D_Relevante[pala] + 1)/(len(palavras_R)+len(PALAVRAS_RI))
    else:
        pro = pro * (1/(len(palavras_R)+len(PALAVRAS_RI)))
pb.append(pro)

PRO = 1
for plvra in ahoi:
    
    if plvra in D_Irrelevante.keys():
        PRO = PRO * (D_Irrelevante[plvra] + 1)/(len(palavras_I)+len(PALAVRAS_RI))
    else:
        PRO = PRO * (1/(len(palavras_R)+len(PALAVRAS_RI)))
pb.append(PRO)

if pb[0] > pb[1]:
    print('Relevante')
else:
    print('Nha') #Irrelevante

Relevante


# Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)


In [17]:
Fato = ['Relevante', 'Irrelevante']
Classificação = ['Verdadeiro', 'Falso']
valores = [[pos_ved,neg_fal],[neg_ved,pos_fal]]

crosstab = pd.DataFrame(np.array(valores), index=Fato, columns = Classificação)

crosstab

,Verdadeiro,Falso
Relevante,0.225,0.135
Irrelevante,0.600,0.040


A tabela acima, apresenta em sua primeira linha, as probabilidades dos tweets classificados como relevantes no excel e que, foram marcados como tais pelo sistema (Relevantes Verdadeiro) ou marcados como irrelevantes pelo sistema (Relevantes Falso). Na segunda linha, se apresenta o mesmo princípio, os tweets classificados como irrelevantes no excel e que foram, marcados como irrelevantes pelo Naive-Bayes (Irrelevantes Verdadeiros) ou marcados como relevantes (Irrelevantes Falsos).

In [30]:
dado = ['Positivo', 'Negativo']
val = [[pos_ved, pos_fal], [neg_ved,neg_fal]]

print('Positivos Verdadeiros {0} %'.format(pos_ved*100)) #marcados como rele sendo rele #positivos verdadeiros
print('Negativos Falsos {0} %'.format(neg_fal*100))  #marcados como irre sendo rele #negativos falsos
print('Negativos Verdadeiros {0} %'.format(neg_ved*100)) #marcado como irre sendo irre #negativos verdadeiros
print('Positivos Falsos {0} %'.format(pos_fal*100)) #marcado como rele sendo irre #positivos falsos

pd.DataFrame(np.array(val), index=dado, columns = Classificação)

Positivos Verdadeiros 22.5 %
Negativos Falsos 13.5 %
Negativos Verdadeiros 60.0 %
Positivos Falsos 4.0 %


,Verdadeiro,Falso
Positivo,0.225,0.040
Negativo,0.600,0.135


___
## Concluindo...

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


### Conclusão:

O classificador Naive-Bayes poderia ser utilizado para inúmeras funções, tal como determinar se a pessoa possui ou não uma certa doença, usando como base de dados, sintomas, ocorrências e a resposta do médico, classificando os examinados em apresentar sintomas da doença, mas não estar doente ou não apresentar sintomas e não ter a doença, etc. Outro exemplo seria utilizar esse sistema para calcular as probabilidades de classificar uma fruta, dada algumas características. O Naive-Bayes também pode ser utilizado no cenário político, em que pode-se analisar a opinião pública e projetar as probabilidades de cada candidato vencer a eleição. Contudo, nesse projeto utilizamos o classificador para identificar e analisar mensagens de opinião sobre um produto, no nosso caso o Netflix.

Após terminar de desenvolver o nosso classificador e testá-lo em vários tweets, foi possível chegar à conclusão de que o classificador criado a partir do Naive-Bayes não é perfeito e apresenta erros. Dentre os 72 classificados como relevantes, 45 foram marcados como realmente relevantes pelo sistema e 27 como não relevantes (61.64% de acerto e 38.36% de erro, olhando apenas os classificados como relevantes por nós). Já dos 128 classificados como irrelevante, 93.70% deles foram marcados corretamente, enquanto que os 6.30% restantes foram marcados como sendo relevantes. Ou seja, para os irrelevantes, o número de Positivos Falsos obtido pelo classificador foi bem pequeno, trazendo certa confiabilidade para ele. Apesar que, como há uma taxa significativa de Negativos Falsos, pode-se dizer por outro lado que esse classificador também não é muito eficiente, já que este apresenta um índice razoável de erros.

O número de Negativos Falsos deve-se principalmente ao fato de haver, nos tweets analisados, mais tweets irrelevantes do que relevantes. Assim, os artigos, preposições e verbos de ligação por exemplo, que aparecem mais e não trazem nenhum conteúdo relevante para análise, aumentam as probabilidades dos tweets de serem irrelevantes. Ademais, dentre os tweets há mensagens com dupla negação e sarcasmo. Como o Naive-Bayes trata cada palavra do tweet individualmente, ele não junta o sentido de duas palavras; logo, quando há dupla negação, ou seja, duas palavras negativas se anulam, ele pode classificar o tweet incoerentemente ao que seria desejado, o que também contribui para o aumento de erros. O mesmo acontece para o sarcasmo, já que algumas palavras possuem duplo sentido, podendo significar duas coisas diferentes. Dessa forma, quando o classificador passar por uma palavra, ele não irá considerar o contexto dela, e sim o fato dela ocorrer. Assim, um tweet considerado relevante que possui uma palavra ambígua ou utiliza sarcasmo, pode ser marcado como irrelevante, já que ele ve a palavra com outro sentido. 

Dessa forma, propomos um plano de expansão possibilite ampliar a análise dos dados, se iniciando por uma melhor limpeza e poda dos dados. Além disso, outra forma que poderia melhorar drasticamente o classificador seria se conseguíssimos juntar palavras, o que também ampliaria a análise, já que algumas palavras ao ficarem sozinhas, perdem parte de seu sentido original. Para melhorar ainda mais, não devemos alimentar a base "Treinamento" automaticamente usando o nosso próprio classificador, pois assim estaríamos transferindo esse erro, fazendo com que ele aumente cada vez mais (como uma "bola de neve"). Isso pode ser explicados pelo fato de que como já há um erro, os valores calculados equivocadamente transfeririam essa falha para frente. O correto seria explorar outras formas, como por exemplo aumentar a base de dados (tweets classificados manualmente por nós que estarão 100% de acordo com o que a empresa considera relevante ou não), possibilitando mais informações, que tornarão o cálculo mais preciso. 

Por fim, defendemos o fato de que esse projeto deve continuar a ser financiado, pois como esse produto é utilizado por milhões de pessoas, quanto mais se souber sobre a opinião dos usuários, melhor serão as melhorias realizadas, trazendo um ambiente cada vez mais satisfatório e de acordo com o interesse do cliente, o que acarretará em mais pessoas felizes, querendo assinar com a empresa e assim por diante, ou seja, um ciclo virtuoso.

**Referêrencias:**
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/